In [85]:
import cvxpy as cp
import numpy as np

## Set up constants

# Load data files
accomodations = np.loadtxt("accomodation.csv", delimiter=",",skiprows=1,usecols=(1,2,3,4,5,6))

## Set up variables
M = 100000000   # Some arbitrarily large term

# Costs per day (in USD)
cost_weekend = [60, 20, 40, 25, 25, 20, 30, 30]  # Fri/Sat
cost_transition = [25, 15, 25, 20, 30, 20, 30, 25]  # Sun/Thu
cost_weekday = [25, 15, 20, 20, 20, 20, 30, 25]  # Mon/Tue/Wed

# Travel costs (matrix in USD)
travel_cost = [
    [M, 40, 65, 20, 50, 30, 50, 60],
    [40, M, 105, 40, 130, 70, 165, 130],
    [65, 105, M, 100, 60, 25, 30, 40],
    [20, 40, 100, M, 86, 20, 66, 75],
    [50, 130, 60, 86, M, 100, 80, 20],
    [30, 70, 25, 20, 100, M, 90, 130],
    [50, 165, 30, 66, 80, 90, M, 40],
    [60, 130, 40, 75, 20, 130, 40, M]
]

transport_time = [
    [M,2.50,4.75,2.00,3.00,2.50,2.75,2.00],
    [2.50,M,6.50,3.00,6.00,6.75,6.50,4.75],
    [4.75,6.50,M,6.00,3.00,5.00,1.50,1.75],
    [2.00,3.00,6.00,M,6.00,2.00,6.00,6.00],
    [3.00,6.00,3.00,6.00,M,6.00,2.00,1.00],
    [2.50,6.75,5.00,2.00,6.00,M,5.50,6.00],
    [2.75,6.50,1.50,6.00,2.00,5.50,M,1.00],
    [2.00,4.75,1.75,6.00,1.00,6.00,1.00,M]	
]

is_coastal = accomodations[:,4]

# Number of cities
num_cities = 8

# Total budget and length of trip
budget = 1500
trip_length = 14

In [86]:
## Decision variables
x = cp.Variable(num_cities, integer=True)  # Total days spent in each city

y_weekday = cp.Variable(num_cities, integer=True)
y_weekend = cp.Variable(num_cities, integer=True)

t = cp.Variable((num_cities, num_cities), boolean=True)  # Travel route binary variables -- traveling from city i to j

u = cp.Variable(num_cities,nonneg=True) # artificial variable for eliminating subtours

In [ ]:
## Create objective

# Objective: Minimize total cost
total_cost = (
    sum(cost_weekend[i] * y_weekend[i] for i in range(num_cities)) +
    sum(cost_weekday[i] * y_weekday[i] for i in range(num_cities)) +
    sum(
        travel_cost[i][j] * t[i, j] 
        for i in range(num_cities) 
        for j in range(num_cities)
    )
)

# Constraints
constraints = [
    y_weekday >= 0, # Nonnegativity constraint
    y_weekend >= 0,
    
    x >= 1, # Each city visited at least once
    
    x <= 3,  # No more than 3 days in a single city
    sum(x) == trip_length,  # Total days equal to trip duration
    sum(
        cost_weekend[i] * y_weekend[i] +
        cost_weekday[i] * y_weekday[i] +
        sum(travel_cost[i][j] * t[i, j] for j in range(num_cities))
        for i in range(num_cities)
    ) <= budget # Budget constraint
]

# Total of each type of days of the week on the trip
constraints.append(sum(y_weekday) == 8)
constraints.append(sum(y_weekend) == 6)

# Spend at least 4 days on the coast
constraints.append(sum(is_coastal[i] * x[i] for i in range(num_cities)) >= 4)

# Spend at least 2 weekend days in Madrid or Barcelona
constraints.append(y_weekend[0] + y_weekend[1] >= 2)

# Travel no more than 6 hours per day
for i in range(num_cities):
    for j in range(num_cities):
        constraints.append(transport_time[i][j] * t[i, j]  <= 6)

# Individual constraints for day breakdown
for i in range(num_cities):
    constraints.append(y_weekday[i] + y_weekend[i] == x[i])

In [88]:
# Each city visited exactly once
for i in range(num_cities): 
    constraints.append(sum(t[i, j] for j in range(num_cities)) == 1)
    constraints.append(sum(t[j, i] for j in range(num_cities)) == 1)
 
# No subtours
for i in range(1,num_cities):
   for j in range(1,num_cities):
       constraints.append(u[i] - u[j] + num_cities*t[i,j] <= num_cities-1)

In [89]:
# Solver
problem = cp.Problem(cp.Minimize(total_cost), constraints)
problem.solve(solver=cp.GUROBI)

# Results
print("Optimal Cost:", problem.value)
print("Days in each city:", [round(val) for val in x.value])
print("Weekdays in each city:", [round(val) for val in y_weekday.value])
print("Weekends in each city:", [round(val) for val in y_weekend.value])
print("Route:")
print(t.value)

Optimal Cost: 560.0
Days in each city: [1, 3, 3, 1, 1, 3, 1, 1]
Weekdays in each city: [1, 2, 3, 1, 0, 0, 0, 1]
Weekends in each city: [0, 1, 0, 0, 1, 3, 1, 0]
Route:
[[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]
